In [1]:
import os
import sagemaker
from sagemaker.tensorflow import TensorFlow

sagemaker_session = sagemaker.Session()

bucket = 'calvinandpogs-ee148'

role = sagemaker.get_execution_role()
print(bucket, role)
print(os.getcwd())

calvinandpogs-ee148 arn:aws:iam::652516965730:role/service-role/AmazonSageMaker-ExecutionRole-20210513T011299
/home/ec2-user/SageMaker/atrw/models/yolov5


## Yolov5s Testing

In [2]:
# TENSORFLOW CONTAINER (PY37)

estimator = TensorFlow(entry_point='test.py',
                        source_dir='./',
                        role=role,
                        instance_count=1,
                        instance_type="ml.g4dn.xlarge",
                        framework_version="2.2",
                        py_version="py37",
                        hyperparameters={
                            'img-size': 1920,
                            'task': 'test',
                            'weights': 'SM_CHANNEL_MODEL/best.pt',
                            'data': 'atrw.yaml',
                            'anno-json': 'data/atrw_detect_center.json',
                            'output-s3': 's3://calvinandpogs-ee148/atrw/out/detection/yolov5/test/',
                            'save-s3': True
                        }
)

In [ ]:
estimator.fit({'model': f's3://{bucket}/models/yolov5/train-full/',
               'annot_test': f's3://{bucket}/atrw/detection/annotations/yolov5-test/ImageSets',
               'label_test': f's3://{bucket}/atrw/detection/annotations/yolov5-test/labels',
               'test': f's3://{bucket}/atrw/detection/test/'})

## Yolov5m Testing

In [7]:
# TENSORFLOW CONTAINER (PY37)

estimator = TensorFlow(entry_point='test.py',
                        source_dir='./',
                        role=role,
                        instance_count=1,
                        instance_type="ml.g4dn.xlarge",
                        framework_version="2.2",
                        py_version="py37",
                        hyperparameters={
                            'img-size': 1920,
                            'task': 'test',
                            'weights': 'SM_CHANNEL_MODEL/best.pt',
                            'data': 'atrw.yaml',
                            'anno-json': 'data/atrw_detect_corner.json',
                            'output-s3': 's3://calvinandpogs-ee148/atrw/out/detection/yolov5/test-m/',
                            'save-s3': True
                        }
)

In [8]:
estimator.fit({'model': f's3://{bucket}/models/yolov5/train-full-m/',
               'annot_test': f's3://{bucket}/atrw/detection/annotations/yolov5-test/ImageSets',
               'label_test': f's3://{bucket}/atrw/detection/annotations/yolov5-test/labels',
               'test': f's3://{bucket}/atrw/detection/test/'})

2021-05-26 23:05:55 Starting - Starting the training job...
2021-05-26 23:06:21 Starting - Launching requested ML instancesProfilerReport-1622070350: InProgress
......
2021-05-26 23:07:21 Starting - Preparing the instances for training......
2021-05-26 23:08:23 Downloading - Downloading input data......
2021-05-26 23:09:21 Training - Downloading the training image...
2021-05-26 23:09:41 Training - Training image download completed. Training in progress.2021-05-26 23:09:37.949997: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:425] Initializing the SageMaker Profiler.
2021-05-26 23:09:37.953471: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:106] SageMaker Profiler is not enabled. The timeline writer thread will not be started, future recorded events will be dropped.
2021-05-26 23:09:38.052589: W tensorflow/core/profiler/internal/smprofiler_timeline.cc:425] Initializing the SageMaker Profiler.
2021-05-26 23:09:40,674 sagemaker-training-toolkit INFO     Imported f

## Testing Subsets

### Fractional Subsets

In [9]:
%%capture --no-stderr

sets = [16, 8, 4, 2]

for set in sets:
    estimator = TensorFlow(entry_point='test.py',
                            source_dir='./',
                            role=role,
                            instance_count=1,
                            instance_type="ml.g4dn.xlarge",
                            framework_version="2.2",
                            py_version="py37",
                            hyperparameters={
                                'img-size': 1920,
                                'task': 'test',
                                'weights': 'SM_CHANNEL_MODEL/best.pt',
                                'data': f'atrw{set}.yaml',
                                'anno-json': 'data/atrw_detect_center.json',
                                'output-s3': f's3://calvinandpogs-ee148/atrw/out/detection/yolov5/frac/test{set}/',
                                'save-s3': True
                            }
    )
    estimator.fit({'model': f's3://{bucket}/models/yolov5/train{set}/',
                   'annot_test': f's3://{bucket}/atrw/detection/annotations/yolov5-test/ImageSets',
                   'label_test': f's3://{bucket}/atrw/detection/annotations/yolov5-test/labels',
                   'test': f's3://{bucket}/atrw/detection/test/'})

### Clustering Split

In [9]:
%%capture --no-stderr

sets = [16, 8, 4, 2]

for set in sets:
    estimator = TensorFlow(entry_point='test.py',
                            source_dir='./',
                            role=role,
                            instance_count=1,
                            instance_type="ml.g4dn.xlarge",
                            framework_version="2.2",
                            py_version="py37",
                            hyperparameters={
                                'img-size': 1920,
                                'task': 'test',
                                'weights': 'SM_CHANNEL_MODEL/best.pt',
                                'data': 'atrw.yaml',
                                'anno-json': 'data/atrw_detect_center.json',
                                'output-s3': f's3://calvinandpogs-ee148/atrw/out/detection/yolov5/bg-split/test/split{set}/',
                                'save-s3': True
                            }
    )
    estimator.fit({'model': f's3://{bucket}/models/yolov5/bg-split/split{set}/',
                   'annot_test': f's3://{bucket}/atrw/detection/annotations/yolov5-test/ImageSets',
                   'label_test': f's3://{bucket}/atrw/detection/annotations/yolov5-test/labels',
                   'test': f's3://{bucket}/atrw/detection/test/'})